**DATASET**

In [ ]:
import pandas as pd

df = pd.read_excel('10Jogos.xlsx', engine='openpyxl', header=0)
df

**Sorteio De Capitães**

In [ ]:
import numpy as np

def sorteia_capitao(df):
    #Sortear 2 Jogadores
    capitao_sorteio = df['Players'].sample(2, random_state=np.random.RandomState())

    capitao1 = capitao_sorteio.iloc[0]
    capitao2 = capitao_sorteio.iloc[1]

    coletes_bola, campo = np.random.choice([capitao1, capitao2], size=2, replace=True)

    resultado = (
        f"Capitão 1 - '{capitao1}'\n"
        f"Capitão 2 - '{capitao2}'\n"
        f"Coletes, Bola - Equipa de {coletes_bola}\n"
        f"Campo - Equipa de {campo}"
    )
    return resultado

print(sorteia_capitao(df))

**Tabela de Classificação**

In [ ]:
import pandas as pd

def classificar_jogadores(df: pd.DataFrame) -> pd.DataFrame:
    #Pesos
    pesos = {
        'Golos': 2, 'Assists': 1, 'Saves': 0.25, 'Ultra-Save' : 3, 'Save de Penalty': 2, 'MVP': 3.5,
        'Amarelos': -0.5, 'Vermelhos': -1, 'Vitórias': 5, 'Derrotas': -2.5, 'Frango' : -2,
        'Cortes Chave': 1.5, 'Penalty Marcado' : 0.5, 'Penalty Falhado' : -2, 'Golo de Livre' : 1,  
        'Jogada Chave' : 1, 'Passe Chave' : 0.75, 'Auto-Golo' : - 1, 'Livre Falhado' : -0.25, 'Erro Chave' : -2,
        'Faltou' : -10, 'Compensa' : 2.5, 'Falhanço' : -2
    }
    
    #Pontuação 
    df['Points'] = (
        df['Golos'] * pesos['Golos'] + df['Assists'] * pesos['Assists'] + df['Falhanço'] * pesos['Falhanço'] + #Golos/Assists
        df['Saves'] * pesos['Saves'] + df['Ultra-Save'] * pesos['Ultra-Save'] + df['Frango'] * pesos['Frango'] + #Saves
        df['Faltou Jogo'] * pesos['Faltou'] + #Penalização
        df['Compensa'] * pesos['Compensa'] + #Compensação
        df['MVP'] * pesos['MVP'] + #MVP
        df['Auto-Golo'] * pesos['Auto-Golo'] + #Auto-Golo
        df['Vitórias'] * pesos['Vitórias'] + df['Derrotas'] * pesos['Derrotas'] + #Vitória7Derrota
        df['Golo de Penalty'] * pesos['Penalty Marcado'] + df['Save de Penalty'] * pesos['Save de Penalty'] + df['Penalty Falhado'] * pesos['Penalty Falhado'] + #Penaltys
        df['Livre Falhado'] * pesos['Livre Falhado'] + df['Golo de Livre'] * pesos['Golo de Livre'] + #Livres
        df['Cartões Amarelos'] * pesos['Amarelos'] + df['Cartões Vermelhos'] * pesos['Vermelhos'] + #Cartões
        df['Cortes Chave'] * pesos['Cortes Chave'] + df['Passe Chave'] * pesos['Passe Chave'] + df['Jogada Chave'] * pesos['Jogada Chave'] + df['Erro Chave'] * pesos['Erro Chave']  #Chave
    )
    
    #Bónus
    df['Bônus'] = 0
    df.loc[df['HUB'] == 'ATA', 'Bônus'] = df['Golos'] * 1
    df.loc[df['HUB'] == 'MED', 'Bônus'] = df['Assists'] * 1
    df.loc[df['HUB'] == 'DEF', 'Bônus'] = df['Cortes Chave'] * 1
    
    df['Points'] += df['Bônus']
    
    #Médias e Percentagens
    df['Win_Rate'] = ((df['Vitórias'] / df['Jogos']) * 100).fillna(0).apply(lambda x: f"{x:.0f}%")
    df['Média de Golos'] = (df['Golos'] / df['Jogos']).apply(lambda x: f"{x:.1f}")
    df['Média de Assists'] = (df['Assists'] / df['Jogos']).apply(lambda x: f"{x:.1f}")
    df['Média de Saves'] = (df['Saves'] / df['Jogos']).apply(lambda x: f"{x:.1f}")
    df['Média de Cortes'] = (df['Cortes Chave'] / df['Jogos']).apply(lambda x: f"{x:.1f}")
    df['Acerto de Penalty'] = (df['Golo de Penalty'] / (df['Golo de Penalty'] + df['Penalty Falhado']) * 100).fillna(0).apply(lambda x: f"{x:.0f}%")
    df['Acerto de Livres'] = (df['Golo de Livre'] / (df['Golo de Livre'] + df['Livre Falhado']) * 100).fillna(0).apply(lambda x: f"{x:.0f}%")
    
    df = df.sort_values(by='Points', ascending=False)
    
    #Ordenar Colunas
    colunas = [
        'Points', 'Players', 'Vitórias', 'Derrotas', 'Jogos', 'Win_Rate', 'MVP',
        'Golos', 'Assists', 'Saves', 'Cortes Chave', 'Média de Golos', 'Média de Assists', 'Média de Saves', 'Média de Cortes','Ultra-Save', 'Frango',
        'Save de Penalty', 'Golo de Penalty', 'Penalty Falhado', 'Acerto de Penalty',
        'Golo de Livre', 'Livre Falhado', 'Acerto de Livres', 'Cartões Amarelos', 'Cartões Vermelhos',
        'Auto-Golo', 'Jogada Chave', 'Passe Chave', 'Erro Chave', 'Falhanço', 'Players'
    ]
    df = df[colunas]
    
    return df

df_ordenado = classificar_jogadores(df)

print(df_ordenado.to_string(index=False))
